In [1]:
#IMPORT THE LIBRARIES

import pandas as pd
import matplotlib.pylab as plt

import types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import os

In [15]:
#FUNCTIONS DEFINED TO UPLOAD/DOWNLOAD TO OBJECT STORE

from ibm_botocore.client import Config
import ibm_boto3
def upload_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.upload_file(Filename=local_file_name, Bucket=credentials['BUCKET'],Key=key)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Uploaded : ' + local_file_name)

from ibm_botocore.client import Config
import ibm_boto3

def download_file_cos(credentials,local_file_name,key):  
    cos = ibm_boto3.client(service_name='s3',
    ibm_api_key_id=credentials['IBM_API_KEY_ID'],
    ibm_service_instance_id=credentials['IAM_SERVICE_ID'],
    ibm_auth_endpoint=credentials['IBM_AUTH_ENDPOINT'],
    config=Config(signature_version='oauth'),
    endpoint_url=credentials['ENDPOINT'])
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'],Key=key,Filename=local_file_name)
    except Exception as e:
        print(Exception, e)
    else:
        print('File Downloaded')

In [5]:
#DOWNLOAD THE 311 DATA WITH THE API CALL
filename = "https://data.cityofnewyork.us/resource/fhrw-4uyv.csv?$limit=100000000&Agency=HPD&$select=created_date,unique_key,complaint_type,incident_zip,incident_address,street_name,address_type,city,resolution_description,borough,latitude,longitude,closed_date,location_type,status"
#100000000
df_311 = pd.read_csv(filename, parse_dates=True)

In [6]:
#DEBUG
df_311.head(3)

,created_date,unique_key,complaint_type,incident_zip,incident_address,street_name,address_type,city,resolution_description,borough,latitude,longitude,closed_date,location_type,status
0,2019-09-28T11:49:21.000,43915077,PAINT/PLASTER,11385.0,1711 HIMROD STREET,HIMROD STREET,ADDRESS,Ridgewood,The following complaint conditions are still o...,QUEENS,40.705446,-73.914524,NaN,RESIDENTIAL BUILDING,Open
1,2019-09-28T19:43:42.000,43915957,APPLIANCE,10458.0,2700 GRAND CONCOURSE,GRAND CONCOURSE,ADDRESS,BRONX,The following complaint conditions are still o...,BRONX,40.867269,-73.893454,NaN,RESIDENTIAL BUILDING,Open
2,2019-09-28T13:55:54.000,43914291,HEAT/HOT WATER,11417.0,86-07 SUTTER AVENUE,SUTTER AVENUE,ADDRESS,Ozone Park,The complaint you filed is a duplicate of a co...,QUEENS,40.675885,-73.850473,NaN,RESIDENTIAL BUILDING,Open


In [7]:
#SERIALIZE BY CONVERTING TO A PKL FILE
df_311.to_pickle('311_ALL.pkl')

In [11]:
!ls -lart

total 570824
drwx------ 1 dsxuser dsxuser      4096 Oct  9 21:05 ..
drwxr-x--- 2 dsxuser dsxuser      4096 Oct  9 21:48 .
-rw-r----- 1 dsxuser dsxuser 584510161 Oct  9 21:48 311_ALL.pkl


In [12]:
#UPLOAD 311 DATA TO OBJECT STORE
upload_file_cos(credentials,'311_ALL.pkl','311_ALL.pkl')

File Uploaded


In [5]:
#GET THE PLUTO DATASET:
!wget https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_18v1.zip

--2019-10-10 13:56:41--  https://www1.nyc.gov/assets/planning/download/zip/data-maps/open-data/nyc_pluto_18v1.zip
Resolving www1.nyc.gov (www1.nyc.gov)... 23.67.220.154, 2600:1404:6800:3a1::1500, 2600:1404:6800:3aa::1500
Connecting to www1.nyc.gov (www1.nyc.gov)|23.67.220.154|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 48263311 (46M) [application/zip]
Saving to: ‘nyc_pluto_18v1.zip’

100%[======================================>] 48,263,311  15.5MB/s   in 3.0s   

2019-10-10 13:56:44 (15.5 MB/s) - ‘nyc_pluto_18v1.zip’ saved [48263311/48263311]



In [6]:
!ls

nyc_pluto_18v1.zip


In [7]:
!unzip nyc_pluto_18v1.zip

Archive:  nyc_pluto_18v1.zip
  inflating: PLUTO_for_WEB/BK_18v1.csv  
  inflating: PLUTO_for_WEB/BX_18v1.csv  
  inflating: PLUTO_for_WEB/MN_18v1.csv  
  inflating: PLUTO_for_WEB/PLUTODD18v1.pdf  
  inflating: PLUTO_for_WEB/PlutoReadme18v1.pdf  
  inflating: PLUTO_for_WEB/QN_18v1.csv  
  inflating: PLUTO_for_WEB/SI_18v1.csv  


In [8]:
!ls

nyc_pluto_18v1.zip  PLUTO_for_WEB


In [9]:
!ls PLUTO_for_WEB

BK_18v1.csv  MN_18v1.csv      PlutoReadme18v1.pdf  SI_18v1.csv
BX_18v1.csv  PLUTODD18v1.pdf  QN_18v1.csv


In [10]:
def convertToPKLandUpload(local_csv_file_name, pklFileName, objectStoreCreds):
    tempDF = pd.read_csv(local_csv_file_name)
    tempDF.to_pickle(pklFileName)
    upload_file_cos(objectStoreCreds,pklFileName,pklFileName)

In [16]:
def uploadCSVtoObjectStoreAsPickle(directory, fileExtension):
    for filename in os.listdir(directory):
        if filename.endswith(fileExtension):
            #DEBUG: NO SIDE EFFECTS NORMALLY
            #print (filename)
            fileName, extension = filename.split(".")
            convertToPKLandUpload(directory+'/' + filename, fileName + '.pkl', credentials)

In [17]:
uploadCSVtoObjectStoreAsPickle("PLUTO_for_WEB", ".csv")

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (20,23,24,26,64) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


File Uploaded : MN_18v1.pkl


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (19,20,22,23,24,64,65,80) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


File Uploaded : SI_18v1.pkl


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (19,20,22,23,24,25,26,64,65,80) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


File Uploaded : BK_18v1.pkl


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (19,20,22,23,64,65,77) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


File Uploaded : QN_18v1.pkl


/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3267: DtypeWarning: Columns (19,20,22,23,64,65,80) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


File Uploaded : BX_18v1.pkl


In [ ]:
#to DOWNLOAD FROM OBJECT STORE
#download_file_cos(credentials,'3112.pkl','311.pkl')
# DOES NOT WORK: client_cred.download_file(Bucket=bucket,Key='311.pkl',Filename='./3112.pkl')
#df = pd.read_pickle('./df_raw_local.pkl')